In [1]:
#Import packages
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess
import pandas as pd
import gensim.corpora as corpora
from sklearn.model_selection import train_test_split
from __future__ import print_function
import pyLDAvis
from pyLDAvis import gensim_models as vis_pack

C:\Users\mlogt\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
C:\Users\mlogt\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#Import dataset
df = pd.read_excel("final_df2.xlsx",index_col=0)

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#Simple preprocess
to_df = {} #Initiate temp dict
for i in range(len(df)): #Loop through each row
    to_df[i] = simple_preprocess(str(df.loc[i,'review']), deacc=True) #Simple preprocess implementation
df['review'] = to_df.values() #Copy the data in temp dict to DataFrame
#Transform data to list and make dictionary (dictionary contains each occuring word)
data = list(df['review']) #Transform data to list
id2word = corpora.Dictionary(data) # Make dictionary, where each word links to an id

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\mlogt\anaconda3\lib\site-packages\pandas\core\common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [4]:
#Train test split
train, test = train_test_split(data,train_size=0.8,random_state=3099) #Split data
train_corpus = [] #Initiate train corpus, corpus converts words into tuples (combi of word and frequency of that word)
for review in train: #Loop through train data, transform each review to ids (corresponding to the dictionairy) and add to the train corpus
    new = id2word.doc2bow(review)
    train_corpus.append(new) 
test_corpus = [] #Initiate test corpus, corpus converts words into tuples (combi of word and frequency of that word)
for review in test: #Loop through test data, transform each review to ids (corresponding to the dictionairy) and add to the test corpus
    new = id2word.doc2bow(review)
    test_corpus.append(new)

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
#Tune model
for n_topics in [6,8,10,12,14,16]: #Loop through possible n_topics
    for n_passes in [1,3,5,7,9]: #Loop through possible n_passes (inner loop; NB: goes through each combination of n_topics & n_passes)
        lda_model = gensim.models.ldamulticore.LdaMulticore(workers=5, #Specify model
                                                            corpus=train_corpus,
                                                            id2word=id2word,
                                                            num_topics=n_topics,
                                                            alpha=0.01,
                                                            eta = 1/n_topics,
                                                            passes=n_passes,
                                                            random_state=3099)
        coherence_model_umass = CoherenceModel(model=lda_model, texts=test, dictionary=id2word,coherence='u_mass') #Calculate coherence (UMass)
        coherence_model_cv = CoherenceModel(model=lda_model, texts=test, dictionary=id2word,coherence='c_v') #Calculate coherence (c_v)
        #Print relevant details
        print('Model built with n_topics:',n_topics) #n_topics
        print('Model built with n_passes:',n_passes) #n_passes
        print('Coherence (U_Mass):',coherence_model_umass.get_coherence()) #Coherence (UMass)
        print('Coherence (c_v):',coherence_model_cv.get_coherence()) #Coherence (c_v)
        print() #White row

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model built with n_topics: 6
Model built with n_passes: 1
Coherence (U_Mass): -2.3959861955957305
Coherence (c_v): 0.3930943012503552

Model built with n_topics: 6
Model built with n_passes: 3
Coherence (U_Mass): -2.4787144120144804
Coherence (c_v): 0.43622279035172146

Model built with n_topics: 6
Model built with n_passes: 5
Coherence (U_Mass): -2.3750660113580624
Coherence (c_v): 0.46066378614526404

Model built with n_topics: 6
Model built with n_passes: 7
Coherence (U_Mass): -2.4273739911258687
Coherence (c_v): 0.4848846290278102

Model built with n_topics: 6
Model built with n_passes: 9
Coherence (U_Mass): -2.490732437357813
Coherence (c_v): 0.4966293664014682

Model built with n_topics: 8
Model built with n_passes: 1
Coherence (U_Mass): -2.4280610613876563
Coherence (c_v): 0.39192367028797925

Model built with n_topics: 8
Model built with n_passes: 3
Coherence (U_Mass): -2.4211591565108765
Coherence (c_v): 0.43637047780777577

Model built with n_topics: 8
Model built with n_pass

In [6]:
#Visualise final model
#Interpretation: Best model has 6 topics, 7 passes
lda_model = gensim.models.ldamulticore.LdaMulticore(workers=5, #Specify final model
                                                    corpus=train_corpus,
                                                    id2word=id2word,
                                                    num_topics=6,
                                                    alpha=0.01,
                                                    eta = 0.1,
                                                    passes=7,
                                                    random_state=3099)
pyLDAvis.enable_notebook() #Allow package to show in notebook
p = vis_pack.prepare(lda_model, test_corpus, id2word) #Set visualisation of final model
p #Show visualisation of final model

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.177764 -0.009745       1        1  32.937956
3     -0.079988 -0.092843       2        1  23.869672
0     -0.078548 -0.038358       3        1  17.558019
2      0.108390  0.094906       4        1  11.199111
4      0.037284 -0.056392       5        1  10.943973
1     -0.164903  0.102432       6        1   3.491269, topic_info=          Term         Freq        Total Category  logprob  loglift
26        goed  7850.000000  7850.000000  Default  30.0000  30.0000
44   luisteren  3746.000000  3746.000000  Default  29.0000  29.0000
41        fijn  1519.000000  1519.000000  Default  28.0000  28.0000
93    afspraak  1408.000000  1408.000000  Default  27.0000  27.0000
30       nemen  3589.000000  3589.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
538        weg    17.689585   229.627639   Topic6  -5.8886   0.7914
475     zitten    18.571152   492.457046   Topic6  -5.8400   0.0771
44   luisteren    20.652249  3746.774858   Topic6  -5.7337  -1.8459
33        tijd    19.212236  3178.248005   Topic6  -5.8060  -1.7536
140        mee    18.592414   956.583685   Topic6  -5.8388  -0.5857

[577 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
137       1  0.494674  aandacht
137       2  0.025697  aandacht
137       3  0.043364  aandacht
137       4  0.332460  aandacht
137       5  0.096365  aandacht
...     ...       ...       ...
964       2  0.310918     zwaar
964       3  0.310918     zwaar
964       4  0.118445     zwaar
964       5  0.059223     zwaar
964       6  0.207279     zwaar

[1361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 3, 5, 2])

In [7]:
#Visualise final model
#Interpretation: Best model has 6 topics, 7 passes
lda_model = gensim.models.ldamulticore.LdaMulticore(workers=5, #Specify final model
                                                    corpus=train_corpus,
                                                    id2word=id2word,
                                                    num_topics=10,
                                                    alpha=0.01,
                                                    eta = 0.1,
                                                    passes=5,
                                                    random_state=3099)
pyLDAvis.enable_notebook() #Allow package to show in notebook
p = vis_pack.prepare(lda_model, test_corpus, id2word) #Set visualisation of final model
p #Show visualisation of final model

C:\Users\mlogt\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.175128 -0.001841       1        1  26.932314
3     -0.059967 -0.041679       2        1  13.245838
0     -0.057515  0.013101       3        1   9.369532
9     -0.069578  0.055069       4        1   8.457163
8     -0.055389  0.049191       5        1   8.138349
7      0.171532 -0.016617       6        1   7.690709
4      0.003470 -0.039599       7        1   7.503423
2      0.099728  0.078796       8        1   7.335476
6     -0.061856 -0.164548       9        1   7.252055
1     -0.145553  0.068126      10        1   4.075140, topic_info=         Term         Freq        Total Category  logprob  loglift
30      nemen  3553.000000  3553.000000  Default  30.0000  30.0000
33       tijd  3161.000000  3161.000000  Default  29.0000  29.0000
41       fijn  1510.000000  1510.000000  Default  28.0000  28.0000
93   afspraak  1343.000000  1343.000000  Default  27.0000  27.0000
26       goed  7837.000000  7837.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
286      jaar    35.917176  1080.574135  Topic10  -5.3350  -0.2038
26       goed    53.203720  7837.269683  Topic10  -4.9421  -1.7923
18      weten    31.148611   719.242844  Topic10  -5.4774   0.0608
10      horen    26.578110   442.199949  Topic10  -5.6361   0.3886
313    helpen    27.554812   822.102869  Topic10  -5.6000  -0.1954

[921 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
137       1  0.275885  aandacht
137       2  0.034080  aandacht
137       3  0.016229  aandacht
137       4  0.021097  aandacht
137       5  0.050308  aandacht
...     ...       ...       ...
964       5  0.164223     zwaar
964       7  0.014929     zwaar
964       8  0.074647     zwaar
964       9  0.029859     zwaar
964      10  0.313518     zwaar

[2947 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 10, 9, 8, 5, 3, 7, 2])